# Tarea 2. NN with LMS, Problema 1

## Fundamentos de inteligencia computacional
## por: Juan Pablo Arango A. & Simón Zapata Caro

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from pydoc import help  # can type in the python console `help(name of function)` to get the documentation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import stats
from IPython.display import display, HTML
from pandas.tools.plotting import scatter_matrix
from matplotlib.artist import setp
import time

plt.rcParams['figure.figsize'] = (10,16)

In [2]:
df1     = pd.io.excel.read_excel("DatosPunto1.xlsx", sheetname=0)
classes = df1.iloc[:,-1].copy()
df1 = df1.iloc[:,0:-2].copy()
#normalización min-max
df1 = (df1-np.min(df1, axis = 0))/(np.max(df1, axis = 0)-np.min(df1, axis = 0))

y = np.array(classes)
X = np.array(df1)

In [3]:
X_original = X

Nsamples = X.shape[0]
Ntrain   = int(0.7*Nsamples)
Nval     = int(0.15*Nsamples)
Ntest    = Nsamples-Ntrain-Nval

X_test = X[(Nsamples-Ntest):Nsamples,:]
y_test = y[(Nsamples-Ntest):Nsamples  ]

X = X[0:(Nsamples-Ntest),:]
y = y[0:(Nsamples-Ntest)  ]

In [4]:
def derivating(fx):
    difcoefs = [1,-1]
    return np.convolve(fx,difcoefs,mode='same')

def clearList(L): #arroja una lista de L elementos vacios, no sabia de qué otra forma hacerla
    listica = []
    for i in range(0,L):
        listica.append([]);
    return listica

def LMS_training(X,y,iteraciones=500,alpha=0.1,de_min=0.01):
    
    ##falta mirar como imprimir Error en tiempo real
    
    w_          = np.random.rand(X.shape[1] + 1,)
    n           = X.shape[0]
    nf          = X.shape[1]
    sw          = 0
    j           = 0
    index       = []
    error_array = []
    

    while j < iteraciones and sw == 0:
        for i in range(0,n):
            g  = np.dot(w_,np.insert(X[i,:],0,1).T)
            e  = g - y[i]
            w_ = w_ - alpha*e*np.insert(X[i,:],0,1)
        predicted = []
        for i in range(0,n):
            predicted.append(Predict(X[i,:],w_))
        g = (y-predicted)
        E = 0.5*np.sum(g*g.T)/y.shape #queda la duda
        
        index.append(j)
        error_array.append(E[0])
        d_error_array = derivating(error_array)
        
        if np.abs(d_error_array[len(d_error_array)-1]) < de_min:
            sw = 1
        
        j = j + 1
    print('Función de error: '+str(E[0])+' iterando ' +str(j)+' veces, con alpha: '+str(alpha))
    
    return [w_,predicted]

def Predict(X,w_): 
    return np.dot(w_,np.insert(X,0,1).T)


### Entrenamiento de la red neuronal usando validación bootstraping para $\alpha = logspace(10^{-5}, 0.5=10^{-0.30102}, N = 30)$

In [5]:
index=np.random.permutation(X.shape[0])
X1    = X[index,:]
y1    = y[index]

X_train = X1[0:Ntrain,:]
X_val   = X1[Ntrain:(Ntrain+Nval),:]
y_train = y1[0:Ntrain  ]
y_val   = y1[Ntrain:(Ntrain+Nval)  ]

alpha = np.logspace(-5,-0.30102,num=30)

In [6]:
wLMS       = clearList(len(alpha))
predicted  = clearList(len(alpha))
predicted2 = clearList(len(alpha))
ecm1       = clearList(len(alpha))
ecm2       = clearList(len(alpha))

for i in range(len(alpha)):
    [wLMS[i],predicted[i]] = LMS_training(X_train,y_train,iteraciones=1000,de_min=0.001,alpha=alpha[i])
    ecm1[i] = np.sqrt(np.sum(((predicted[i]-y_train)*(predicted[i]-y_train)))/y_train.shape)[0]
    
    p_i = []
    for j in range(0,X_val.shape[0]):
        p_i.append(Predict(X_val[j,:],wLMS[i]))
    predicted2[i] = p_i
    ecm2[i] = np.sqrt(np.sum(((predicted2[i]-y_val)*(predicted2[i]-y_val)))/y_val.shape)[0]



Función de error: 7.098578201791492 iterando 1000 veces, con alpha: 1e-05
Función de error: 6.673252349813438 iterando 1000 veces, con alpha: 1.4522246129615342e-05
Función de error: 6.282327593392731 iterando 917 veces, con alpha: 2.108956326491282e-05
Función de error: 6.150924584910585 iterando 738 veces, con alpha: 3.0626782849915814e-05
Función de error: 6.009747765591327 iterando 622 veces, con alpha: 4.447696787047604e-05
Función de error: 5.978808181176455 iterando 496 veces, con alpha: 6.459054745140466e-05
Función de error: 5.854437754941373 iterando 378 veces, con alpha: 9.379998277358975e-05
Función de error: 5.836716257990311 iterando 309 veces, con alpha: 0.00013621864367917524
Función de error: 5.7960507731477335 iterando 262 veces, con alpha: 0.00019782006709513538
Función de error: 5.718561825214475 iterando 219 veces, con alpha: 0.00028727917037325835
Función de error: 5.637871125464687 iterando 245 veces, con alpha: 0.00041719388200721575
Función de error: 5.55647859

In [7]:
print('para cada topologia de red')
d = {'alpha': alpha,'error de entrenamiento': ecm1, 'error de validación' : ecm2}
df = pd.DataFrame(data=d)
df

para cada topologia de red


,alpha,error de entrenamiento,error de validación
0,0.000010,3.767911,3.913930
1,0.000015,3.653287,3.835964
2,0.000021,3.544666,3.721764
3,0.000031,3.507399,3.673115
4,0.000044,3.466914,3.613401
5,0.000065,3.457979,3.600316
6,0.000094,3.421823,3.545076
7,0.000136,3.416641,3.531592
8,0.000198,3.404718,3.507562
9,0.000287,3.381882,3.469546


In [8]:
np.log(0.5)/np.log(10)

-0.30102999566398114